In [17]:
import pandas as pd
import numpy as np

# Load the enhanced bond pairs data
df = pd.read_parquet('historical g spread/bond_z.parquet')
print(f"✅ Loaded enhanced bond pairs: {df.shape}")
print(f"Columns: {df.shape[1]} total")
df.info()

✅ Loaded enhanced bond pairs: (12246, 41)
Columns: 41 total
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12246 entries, 0 to 12245
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Security_1             12246 non-null  object  
 1   Security_2             12246 non-null  object  
 2   Last_Spread            12246 non-null  float64 
 3   Z_Score                12246 non-null  float64 
 4   Max                    12246 non-null  float64 
 5   Min                    12246 non-null  float64 
 6   Last_vs_Max            12246 non-null  float64 
 7   Last_vs_Min            12246 non-null  float64 
 8   Percentile             12246 non-null  float64 
 9   XCCY                   12246 non-null  float64 
 10  Custom_Sector_1        12246 non-null  object  
 11  Custom_Sector_2        12246 non-null  object  
 12  Rating_1               12246 non-null  object  
 13  Rating_2               12246 no

In [18]:
def filter_df(df, filters):
    """
    Flexible DataFrame filtering with nested AND/OR/NOT logic and rich operators.
    - filters: dict, see examples below.
    Returns: filtered DataFrame
    """
    def parse_filter(f):
        if isinstance(f, dict):
            # Logical operators
            if "AND" in f:
                return np.logical_and.reduce([parse_filter(sub) for sub in f["AND"]])
            if "OR" in f:
                return np.logical_or.reduce([parse_filter(sub) for sub in f["OR"]])
            if "NOT" in f:
                return ~parse_filter(f["NOT"])
            # Column filter(s)
            masks = []
            for col, cond in f.items():
                if col in ("AND", "OR", "NOT"):
                    continue
                if isinstance(cond, dict):
                    # Operator-based
                    for op, val in cond.items():
                        masks.append(apply_op(df[col], op, val))
                elif isinstance(cond, list):
                    # List means "in"
                    masks.append(df[col].isin(cond))
                else:
                    # Scalar means "=="
                    masks.append(df[col] == cond)
            return np.logical_and.reduce(masks)
        else:
            raise ValueError("Filter must be a dict (or nested dicts for AND/OR/NOT)")
    
    def apply_op(series, op, val):
        if op == "==": return series == val
        if op == "!=": return series != val
        if op == ">": return series > val
        if op == "<": return series < val
        if op == ">=": return series >= val
        if op == "<=": return series <= val
        if op == "in": return series.isin(val)
        if op == "not in": return ~series.isin(val)
        if op == "contains": return series.astype(str).str.contains(val, na=False)
        if op == "not contains": return ~series.astype(str).str.contains(val, na=False)
        if op == "between": return (series >= val[0]) & (series <= val[1])
        if op == "not between": return (series < val[0]) | (series > val[1])
        if op == "isnull": return series.isnull()
        if op == "notnull": return series.notnull()
        raise ValueError(f"Unknown operator: {op}")
    
    mask = parse_filter(filters)
    return df[mask]